In [135]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
import random

In [136]:
data = pd.read_csv('superstore_final.csv',encoding= 'unicode_escape')

In [137]:
data.head(5)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Category,Sub-Category,ProductName,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,Rating
0,42433,AG-2011-2040,1/1/2014,6/1/2014,Standard Class,8,Toby Braunhardt,Consumer,Constantine,Constantine,...,Office Supplies,Storage,"Tenex Lockers, Blue",408.300,2,0.0,106.140,35.46,Medium,4.0
1,22253,IN-2011-47883,1/1/2014,8/1/2014,Standard Class,9,Joseph Holt,Consumer,Wagga Wagga,New South Wales,...,Office Supplies,Storage,"Tenex Lockers, Blue",120.366,3,0.1,36.036,9.72,Medium,4.5
2,48883,HU-2011-1220,1/1/2014,5/1/2014,Second Class,12,Annie Thurman,Consumer,Budapest,Budapest,...,Office Supplies,Storage,"Tenex Lockers, Blue",66.120,4,0.0,29.640,8.17,High,4.0
3,11731,IT-2011-3647632,1/1/2014,5/1/2014,Second Class,20,Eugene Moren,Home Office,Stockholm,Stockholm,...,Office Supplies,Storage,"Tenex Lockers, Blue",44.865,3,0.5,-26.055,4.82,High,4.0
4,22255,IN-2011-47883,1/1/2014,8/1/2014,Standard Class,24,Patrick O'Donnell,Consumer,Dhaka,Dhaka,...,Office Supplies,Storage,"Tenex Lockers, Blue",113.670,5,0.1,37.770,4.70,Medium,4.0


In [138]:
data.count()

Row ID            51290
Order ID          51290
Order Date        51290
Ship Date         51290
Ship Mode         51290
Customer ID       51290
Customer Name     51290
Segment           51290
City              51290
State             51290
Country           51290
Postal Code       11423
Market            51290
Region            51290
ProductID         51290
Category          51290
Sub-Category      51290
ProductName       51290
Sales             51290
Quantity          51290
Discount          51290
Profit            51290
Shipping Cost     51290
Order Priority    51290
Rating            51290
dtype: int64

In [139]:
print(data['ProductID'].unique().size)
print(data['ProductName'].unique().size)

416
413


In [140]:
data['Customer ID'].unique().size

3446

In [141]:
data.isnull().sum()

Row ID                0
Order ID              0
Order Date            0
Ship Date             0
Ship Mode             0
Customer ID           0
Customer Name         0
Segment               0
City                  0
State                 0
Country               0
Postal Code       39867
Market                0
Region                0
ProductID             0
Category              0
Sub-Category          0
ProductName           0
Sales                 0
Quantity              0
Discount              0
Profit                0
Shipping Cost         0
Order Priority        0
Rating                0
dtype: int64

In [142]:
data.columns = [c.replace(' ', '_') for c in data.columns]

In [143]:
data.columns

Index(['Row_ID', 'Order_ID', 'Order_Date', 'Ship_Date', 'Ship_Mode',
       'Customer_ID', 'Customer_Name', 'Segment', 'City', 'State', 'Country',
       'Postal_Code', 'Market', 'Region', 'ProductID', 'Category',
       'Sub-Category', 'ProductName', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Shipping_Cost', 'Order_Priority', 'Rating'],
      dtype='object')

In [144]:
# checking Order_Date  data is one full year because it's better to use a metric per Months or Years in RFM
data['Order_Date'] = pd.to_datetime(data['Order_Date'])
print(data.dtypes)
print(data['Order_Date'].min())
print(data['Order_Date'].max())
print(data.shape)

Row_ID                     int64
Order_ID                  object
Order_Date        datetime64[ns]
Ship_Date                 object
Ship_Mode                 object
Customer_ID                int64
Customer_Name             object
Segment                   object
City                      object
State                     object
Country                   object
Postal_Code              float64
Market                    object
Region                    object
ProductID                  int64
Category                  object
Sub-Category              object
ProductName               object
Sales                    float64
Quantity                   int64
Discount                 float64
Profit                   float64
Shipping_Cost            float64
Order_Priority            object
Rating                   float64
dtype: object
2014-01-01 00:00:00
2014-12-31 00:00:00
(51290, 25)


## Recency

In [145]:
cust_max_purchase = data.groupby('Customer_ID').Order_Date.max().reset_index()

In [146]:
cust_max_purchase.columns = ['Customer_ID','MaxPurchaseDate']

In [147]:
cust_max_purchase.head()

,Customer_ID,MaxPurchaseDate
0,1,2014-08-01
1,2,2014-11-11
2,4,2014-11-15
3,5,2014-11-04
4,6,2014-11-08


In [148]:
cust_max_purchase['Recency'] = (cust_max_purchase['MaxPurchaseDate'].max() - cust_max_purchase['MaxPurchaseDate']).dt.days

In [149]:
# get unique cust_id's
data_user = pd.DataFrame(data['Customer_ID'].unique())
data_user.columns = ['Customer_ID']
data_user.count()

Customer_ID    3446
dtype: int64

In [150]:
data_user = pd.merge(data_user, cust_max_purchase[['Customer_ID','Recency']], on='Customer_ID')

In [151]:
data_user.head()

,Customer_ID,Recency
0,8,109
1,9,98
2,12,15
3,20,2
4,24,13


In [152]:
data_user.Recency.describe()

count    3446.000000
mean       62.494486
std        78.279414
min         0.000000
25%         7.000000
50%        31.000000
75%        96.000000
max       361.000000
Name: Recency, dtype: float64

## Frequency

In [153]:
frequency_df = data.groupby(by=['Customer_ID'], as_index=False)['Order_Date'].count()
frequency_df.columns = ['Customer_ID','Frequency']
frequency_df.head()

,Customer_ID,Frequency
0,1,2
1,2,10
2,4,2
3,5,3
4,6,1


## Monetary

In [154]:
monetary_df = data.groupby(by='Customer_ID',as_index=False).agg({'Sales': 'sum'})
monetary_df.columns = ['Customer_ID','Monetary']
monetary_df.head()

,Customer_ID,Monetary
0,1,185.0634
1,2,3741.2580
2,4,1145.4220
3,5,360.1560
4,6,40.9700


In [155]:
#merge recency dataframe with frequency dataframe
temp_df = data_user.merge(frequency_df,on='Customer_ID')
temp_df.head()

,Customer_ID,Recency,Frequency
0,8,109,8
1,9,98,3
2,12,15,44
3,20,2,13
4,24,13,65


In [156]:

#merge with monetary dataframe to get a table with the 3 columns
rfm_df = temp_df.merge(monetary_df,on='Customer_ID')
#use CustomerID as index
rfm_df.set_index('Customer_ID',inplace=True)
#check the head
rfm_df.head()

,Recency,Frequency,Monetary
Customer_ID,,,
8,109,8,935.77550
9,98,3,674.94600
12,15,44,9386.25960
20,2,13,1783.23420
24,13,65,12592.59918


In [157]:
customers_rank = rfm_df
# Create a new column that is the rank of the value of coverage in ascending order
customers_rank['Rank'] = customers_rank['Monetary'].rank(ascending=0)

customers_rank.head()

,Recency,Frequency,Monetary,Rank
Customer_ID,,,,
8,109,8,935.77550,2032.0
9,98,3,674.94600,2254.0
12,15,44,9386.25960,383.0
20,2,13,1783.23420,1553.0
24,13,65,12592.59918,234.0


In [158]:
customers_rank.sort_values('Rank',ascending=True)

,Recency,Frequency,Monetary,Rank
Customer_ID,,,,
3437,2,251,57254.23180,1.0
3494,2,201,53866.47780,2.0
682,0,114,46711.17370,3.0
3481,4,119,44842.35790,4.0
638,0,152,41958.80298,5.0
...,...,...,...,...
1814,305,1,3.81600,3442.0
3724,103,1,3.00000,3443.0
1224,39,1,2.54400,3444.0


## RFM Quartiles

In [159]:
quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
quantiles

,Recency,Frequency,Monetary,Rank
0.25,7.0,2.0,366.73155,862.25
0.50,31.0,6.0,1416.73250,1723.50
0.75,96.0,19.0,4694.64617,2584.75


In [160]:
quantiles.to_dict()

{'Recency': {0.25: 7.0, 0.5: 31.0, 0.75: 96.0},
 'Frequency': {0.25: 2.0, 0.5: 6.0, 0.75: 19.0},
 'Monetary': {0.25: 366.73154999999997,
  0.5: 1416.7325,
  0.75: 4694.646169999999},
 'Rank': {0.25: 862.25, 0.5: 1723.5, 0.75: 2584.75}}

In [161]:
# Arguments (x = value, p = recency, monetary_value, frequency, d = quartiles dict)
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [162]:
#create rfm segmentation table
rfm_segmentation = rfm_df
rfm_segmentation['R_Quartile'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
rfm_segmentation['M_Quartile'] = rfm_segmentation['Monetary'].apply(FMScore, args=('Monetary',quantiles,))

In [163]:
rfm_segmentation.head()

,Recency,Frequency,Monetary,Rank,R_Quartile,F_Quartile,M_Quartile
Customer_ID,,,,,,,
8,109,8,935.77550,2032.0,1,3,2
9,98,3,674.94600,2254.0,1,2,2
12,15,44,9386.25960,383.0,3,4,4
20,2,13,1783.23420,1553.0,4,3,3
24,13,65,12592.59918,234.0,3,4,4


In [164]:
rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \
                            + rfm_segmentation.F_Quartile.map(str) \
                            + rfm_segmentation.M_Quartile.map(str)
rfm_segmentation.head()

,Recency,Frequency,Monetary,Rank,R_Quartile,F_Quartile,M_Quartile,RFMScore
Customer_ID,,,,,,,,
8,109,8,935.77550,2032.0,1,3,2,132
9,98,3,674.94600,2254.0,1,2,2,122
12,15,44,9386.25960,383.0,3,4,4,344
20,2,13,1783.23420,1553.0,4,3,3,433
24,13,65,12592.59918,234.0,3,4,4,344


In [165]:
print("Best Customers: ",len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']))
print('Loyal Customers: ',len(rfm_segmentation[rfm_segmentation['F_Quartile']==4]))
print("Big Spenders: ",len(rfm_segmentation[rfm_segmentation['M_Quartile']==4]))
print('Almost Lost: ', len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']))
print('Lost Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']))
print('Lost Cheap Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='111']))

Best Customers:  597
Loyal Customers:  827
Big Spenders:  862
Almost Lost:  11
Lost Customers:  0
Lost Cheap Customers:  442


In [166]:
rfm_segmentation=rfm_segmentation.reset_index()

In [167]:
rfm_segmentation.loc[rfm_segmentation['RFMScore']=='444','Type']= 'Top'
rfm_segmentation.loc[rfm_segmentation['R_Quartile']==1,'Type']= 'Lost'
rfm_segmentation.loc[(rfm_segmentation['R_Quartile']==2) | (rfm_segmentation['R_Quartile']==3) |(rfm_segmentation['F_Quartile']==3) | (rfm_segmentation['F_Quartile']==2) | (rfm_segmentation['M_Quartile']==3) | (rfm_segmentation['M_Quartile']==2) ,'Type']= 'Regular'


In [168]:
## Adding segments to the database
data=pd.merge(data,rfm_segmentation, how ='left', left_on='Customer_ID', right_on='Customer_ID')

In [169]:
top = data[data['Type']=='Top']
lost = data[data['Type']=='Lost']
regular = data[data['Type']=='Regular']

#best_customer_comp.to_csv("C:/Users/yashr/OneDrive/Desktop/best_customer.csv", index=False)

In [184]:
reg_lost = pd.concat([regular,lost])
reg_top_lost =  pd.concat([regular,lost,top])
reg_lost.to_csv("reg_lost.csv", index=False)
top.to_csv("top.csv", index=False)
reg_top_lost.to_csv("reg_top_lost.csv", index=False)

In [171]:
plot_data = [
    go.Scatter(
        x=rfm_segmentation.query("RFMScore == '111'")['Frequency'],
        y=rfm_segmentation.query("RFMScore == '111'")['Monetary'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=rfm_segmentation.query("RFMScore == '222'")['Frequency'],
        y=rfm_segmentation.query("RFMScore == '222'")['Monetary'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=rfm_segmentation.query("RFMScore == '333' ")['Frequency'],
        y=rfm_segmentation.query("RFMScore == '333'")['Monetary'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'orange',
            opacity= 0.9
           )
    ),
    go.Scatter(
        x=rfm_segmentation.query("RFMScore == '444'")['Frequency'],
        y=rfm_segmentation.query("RFMScore == '444'")['Monetary'],
        mode='markers',
        name='Most',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    )
]

plot_layout = go.Layout(
        yaxis= {'title': "Monetary"},
        xaxis= {'title': "Frequency"},
        title='Frequent customers generate most revenue'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [172]:
plot_data = [
    go.Scatter(
        x=rfm_segmentation.query("RFMScore == '111'")['Recency'],
        y=rfm_segmentation.query("RFMScore == '111'")['Monetary'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=rfm_segmentation.query("RFMScore == '222'")['Recency'],
        y=rfm_segmentation.query("RFMScore == '222'")['Monetary'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=rfm_segmentation.query("RFMScore == '333' ")['Recency'],
        y=rfm_segmentation.query("RFMScore == '333'")['Monetary'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'orange',
            opacity= 0.9
           )
    ),
    go.Scatter(
        x=rfm_segmentation.query("RFMScore == '444'")['Recency'],
        y=rfm_segmentation.query("RFMScore == '444'")['Monetary'],
        mode='markers',
        name='Most',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    )
]

plot_layout = go.Layout(
        yaxis= {'title': "Monetary"},
        xaxis= {'title': "Recency"},
        title='Recent customers generate the most revenue'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [173]:
plot_data = [
    go.Scatter(
        x=rfm_segmentation.query("RFMScore == '111'")['Frequency'],
        y=rfm_segmentation.query("RFMScore == '111'")['Recency'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=rfm_segmentation.query("RFMScore == '222'")['Frequency'],
        y=rfm_segmentation.query("RFMScore == '222'")['Recency'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=rfm_segmentation.query("RFMScore == '333' ")['Frequency'],
        y=rfm_segmentation.query("RFMScore == '333'")['Recency'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'orange',
            opacity= 0.9
           )
    ),
    go.Scatter(
        x=rfm_segmentation.query("RFMScore == '444'")['Frequency'],
        y=rfm_segmentation.query("RFMScore == '444'")['Recency'],
        mode='markers',
        name='Most',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    )
]

plot_layout = go.Layout(
        yaxis= {'title': "Recency"},
        xaxis= {'title': "Frequency"},
        title='Frequent Customers are the Most Recent customers'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

## Customer Lifetime Values

In [174]:
data_group=data.groupby('Customer_ID').agg({'Order_Date': lambda date: (date.max() - date.min()).days,
                                        'Order_ID': lambda num: len(num),
                                        'Quantity': lambda quant: quant.sum(),
                                        'Sales': lambda price: price.sum(),
                                        'Profit':lambda profit: profit.sum()})
data_group.head(5)

,Order_Date,Order_ID,Quantity,Sales,Profit
Customer_ID,,,,,
1,151,2,8,185.0634,34.6734
2,238,10,33,3741.2580,866.5780
4,169,2,8,1145.4220,211.5220
5,87,3,10,360.1560,57.2124
6,0,1,1,40.9700,10.6522


### AOV

In [175]:
data_group['AOV']=data_group['Sales']/data_group['Order_ID']

In [176]:
data_group.head(5)

,Order_Date,Order_ID,Quantity,Sales,Profit,AOV
Customer_ID,,,,,,
1,151,2,8,185.0634,34.6734,92.5317
2,238,10,33,3741.2580,866.5780,374.1258
4,169,2,8,1145.4220,211.5220,572.7110
5,87,3,10,360.1560,57.2124,120.0520
6,0,1,1,40.9700,10.6522,40.9700


### Purchase Frequency

In [177]:
purchase_frequency=sum(data_group['Order_ID'])/data_group.shape[0]

### Repeat Rate and Churn Rate

In [178]:
# Repeat Rate
repeat_rate=data_group[data_group.Order_ID > 1].shape[0]/data_group.shape[0]

In [179]:
#Churn Rate
churn_rate=1-repeat_rate

In [180]:
purchase_frequency,repeat_rate,churn_rate

(14.883923389437028, 0.857225769007545, 0.142774230992455)

### Customer Lifetime Value
CLTV = ((Average Order Value x Purchase Frequency)/Churn Rate) x Profit margin.

In [181]:
# Customer Value
data_group['CLV']=(data_group['AOV']*purchase_frequency)/churn_rate

#Customer Lifetime Value
data_group['Cust_Lifetime_Value']=data_group['CLV']*data_group['Profit']

data_group.head()

,Order_Date,Order_ID,Quantity,Sales,Profit,AOV,CLV,Cust_Lifetime_Value
Customer_ID,,,,,,,,
1,151,2,8,185.0634,34.6734,92.5317,9646.241652,3.344680e+05
2,238,10,33,3741.2580,866.5780,374.1258,39001.854232,3.379815e+07
4,169,2,8,1145.4220,211.5220,572.7110,59703.957703,1.262870e+07
5,87,3,10,360.1560,57.2124,120.0520,12515.176992,7.160233e+05
6,0,1,1,40.9700,10.6522,40.9700,4271.039228,4.549596e+04


In [182]:
data_group=data_group.reset_index()

In [183]:
data_group.to_csv("data_group.csv", index=False)
data.to_csv("data.csv", index=False)